### 목록
* 준비
    * [사용할 라이브러리 불러오기](#lib)
    * [엘라스틱서치 환경 설정하기](#es_setting)
    * [엘라스틱서치에서 데이터 조회하기](#es_load)
    * [조회한 데이터를 테이블 형태로 만들기](#df)
* Bucket Aggregation
    * [Date Histogram Aggregation](#dh)
    * [Date Range Aggregation](#dr)
    * [Histogram Aggregation](#ha)
    * [Range Aggregation](#ra)
    * [Terms Aggregation](#ta)
    * [Filters Aggregation](#fa)
---

### 준비

<a name='lib'></a>
#### 사용할 라이브러리 불러오기

In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
import datetime

<a name='es_setting'></a>
#### 엘라스틱서치 환경 설정하기

In [2]:
# 적당한 값을 넣지 않으면 오류가 납니다
es = Elasticsearch()
es_index = 'test'
es_type = 'test'

<a name='es_load'></a>
#### 엘라스틱서치에서 데이터 조회하기

In [3]:
x = es.search(
      index=es_index, 
      doc_type=es_type, 
      body={
        '_source' : ['결제카드','고객나이', '구매사이트', '상품가격', '주문시간'],
        'query' : {
            'match_all' : {}
        },
        'size' : 50
      }
)

<a name='df'></a>
#### 조회한 데이터를 테이블 형태로 만들기

In [4]:
df = pd.DataFrame([x['hits']['hits'][idx]['_source'] for idx, _ in enumerate(x['hits']['hits'])])
df

,결제카드,고객나이,구매사이트,상품가격,주문시간
0,우리,36,g마켓,6000,2017-11-04T04:39:24
1,우리,28,옥션,16000,2017-11-07T02:27:49
2,하나,45,11번가,13000,2017-11-01T21:10:05
3,국민,40,쿠팡,25000,2017-11-12T11:30:19
4,우리,40,11번가,10000,2017-11-12T18:11:58
5,우리,38,11번가,25000,2017-11-08T22:29:44
6,국민,45,위메프,13000,2017-11-07T01:22:52
7,국민,20,위메프,15000,2017-11-11T06:25:42
8,국민,34,g마켓,9000,2017-11-09T01:14:04
9,국민,33,g마켓,5000,2017-11-02T20:24:25


### Bucket Aggregation

<a name='dh'></a>
#### Date Histogram Aggregation

: Date Field를 일정 간격으로 나눈 구간으로 Bucket(혹은 Group) 생성

In [5]:
new_df = df.copy()
new_df.index = pd.to_datetime(new_df['주문시간'])
new_df_rs = new_df.resample('D').count().iloc[:, 1:2]
new_df_rs = pd.DataFrame({'주문시간' : new_df_rs.index, '개수' : new_df_rs.values.ravel()})
new_df_rs[['주문시간', '개수']]

,주문시간,개수
0,2017-11-01,5
1,2017-11-02,4
2,2017-11-03,3
3,2017-11-04,3
4,2017-11-05,1
5,2017-11-06,3
6,2017-11-07,6
7,2017-11-08,5
8,2017-11-09,7
9,2017-11-10,3


<a name="dr"></a>
#### Date Range Aggregation
:  각 Date 구간의 시작과 끝점을 설정하여 Bucket 생성한다

##### 11월1일 ~ 11월3일 구간

In [6]:
new_df_rs = new_df['2017-11-01' : '2017-11-03'].resample('D').count().iloc[:, 1:2]
new_df_rs = pd.DataFrame({'주문시간' : new_df_rs.index, '개수' : new_df_rs.values.ravel()})
new_df_rs[['주문시간', '개수']]

,주문시간,개수
0,2017-11-01,5
1,2017-11-02,4
2,2017-11-03,3


In [7]:
new_df['2017-11-01' : '2017-11-03'].count()[0]

12

##### 11월4일 ~ 11월10일 구간

In [8]:
new_df_rs = new_df['2017-11-04' : '2017-11-10'].resample('D').count().iloc[:, 1:2]
new_df_rs = pd.DataFrame({'주문시간' : new_df_rs.index, '개수' : new_df_rs.values.ravel()})
new_df_rs[['주문시간', '개수']]

,주문시간,개수
0,2017-11-04,3
1,2017-11-05,1
2,2017-11-06,3
3,2017-11-07,6
4,2017-11-08,5
5,2017-11-09,7
6,2017-11-10,3


In [9]:
new_df['2017-11-04' : '2017-11-10'].count()[0]

28

<a name="ha"></a>
#### Histogram Aggregation
: Number Field의 각 구간으로 Bucket 생성한다

In [10]:
bins = range(10, 51, 10)
group_names = ['10대', '20대', '30대', '40대']
df['연령대'] = pd.cut(df['고객나이'], bins, labels=group_names)
df_gb = df.groupby(by='연령대').count().iloc[:, 1:2]
df_gb = pd.DataFrame({'연령대' : df_gb.index, '개수' : df_gb.values.ravel()})
df_gb[['연령대', '개수']]

,연령대,개수
0,10대,3
1,20대,15
2,30대,20
3,40대,12


<a name="ra"></a>
#### Range Aggregation
: 옵션으로 설정한 Number Field의 각 구간으로 Bucket 생성한다

* 상품가격이 만원 이하 : [0, 10,000)
* 상품가격이 이만원 이하 : [10,000 , 20000)
* 상품가격이 삼만원 이하 : [20,000 , 30000)

In [11]:
bins = range(0, 30001, 10000)
group_names = ['만원 이하', '이만원 이하', '삼만원 이하']
df['가격대'] = pd.cut(df['상품가격'], bins, labels=group_names)
df_gb = df.groupby(by='가격대').count().iloc[:, 1:2]
df_gb = pd.DataFrame({'가격대' : df_gb.index, '개수' : df_gb.values.ravel()})
df_gb[['가격대', '개수']]

,가격대,개수
0,만원 이하,15
1,이만원 이하,20
2,삼만원 이하,15


<a name="ta"></a>
#### Terms Aggregation
: 선택한 Term를 기준으로 Bucket 생성한다

* 카드별 결제 건수가 
* 가장 많은 2개의 카드 별
* 최대 결제 금액을 구하시오

In [12]:
# 카드별 결제 건수가...
card = df.groupby(by='결제카드').count().iloc[:, 1:2]
card_df = pd.DataFrame({'결제카드' : card.index, '개수' : card.values.ravel()})
card_df[['결제카드', '개수']]

,결제카드,개수
0,국민,18
1,롯데,5
2,삼성,2
3,시티,2
4,신한,2
5,우리,16
6,하나,5


In [13]:
# ,,,가장 많은 2개의 카드...
card_sort = card_df.sort_values(by='개수', ascending=False)[:2]
card_sort[['결제카드', '개수']]

,결제카드,개수
0,국민,18
5,우리,16


In [14]:
# ...최대 결제 금액을 구하시오
max(df[df['결제카드'] == '우리']['상품가격'])

29000

In [15]:
# ...최대 결제 금액을 구하시오
max(df[df['결제카드'] == '국민']['상품가격'])

25000

<a name="fa"></a>
#### Filters Aggregation
: Query String으로 작성한 조건 만족하는 Bucket 생성한다

* 구매 사이트가 옥션, 티몬인 사람들의
* 구매 사이트별 최소 구매가격을 구하시오

In [16]:
# 구매 사이트가 옥션, 티몬인 사람들의...
df_filter = df[df['구매사이트'].str.contains('옥션|티몬')]
df_filter

,결제카드,고객나이,구매사이트,상품가격,주문시간,연령대,가격대
1,우리,28,옥션,16000,2017-11-07T02:27:49,20대,이만원 이하
14,우리,37,옥션,5000,2017-11-04T03:14:45,30대,만원 이하
16,국민,31,옥션,25000,2017-11-03T20:26:41,30대,삼만원 이하
17,롯데,23,옥션,14000,2017-11-07T08:47:55,20대,이만원 이하
29,신한,45,티몬,29000,2017-11-07T10:55:29,40대,삼만원 이하
35,롯데,21,옥션,28000,2017-11-12T19:42:22,20대,삼만원 이하
38,우리,36,티몬,24000,2017-11-06T02:16:54,30대,삼만원 이하
48,하나,44,옥션,14000,2017-11-10T14:42:32,40대,이만원 이하
49,우리,39,티몬,15000,2017-11-09T16:30:05,30대,이만원 이하


In [17]:
# ...구매사이트별 최소 구매고객을 구하시오
min(df_filter[df_filter['구매사이트']=='옥션']['상품가격'])

5000

In [18]:
# ...구매사이트별 최소 구매고객을 구하시오
min(df_filter[df_filter['구매사이트']=='티몬']['상품가격'])

15000